In [170]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import calendar


In [409]:
class PowerCalendar:
    
    def __init__(self, iso = 'PJM', peak_type = 'onpeak', period = '2018-2-3'):
        
        self.iso = iso
        self.peak_type = peak_type
        
        if '-' in period:
            self.start_date = datetime.datetime.strptime(period, '%Y-%m-%d')
            self.end_date  = datetime.datetime.strptime(period, '%Y-%m-%d')
            
        elif period[-1] == 'A':
            self.start_date = datetime.datetime.strptime(period[:4] + '-1-1', '%Y-%m-%d')
            self.end_date = datetime.datetime.strptime(period[:4] + '-12-31', '%Y-%m-%d')
            
        elif period[-2] == 'Q':
            self.start_date = datetime.datetime(int(period[:4]), 3 * int(period[-1]) - 2, 1)
            self.end_date = datetime.datetime(int(period[:4]), 3 * int(period[-1]) - 2, 1) + relativedelta(months=3, days=-1)
            
        else:
            self.start_date = datetime.date(int(period[:4]), list(calendar.month_abbr).index(period[-3:]), 1)
            self.end_date = datetime.date(int(period[:4]) + 1, 1 , 1) - datetime.timedelta(days=1) if \
            list(calendar.month_abbr).index(period[-3:]) == 12 \
            else datetime.date(int(period[:4]) ,list(calendar.month_abbr).index(period[-3:]) + 1 , 1) \
            - datetime.timedelta(days=1)
            
        self.duration = (self.end_date - self.start_date ).days + 1
        
        self.is_western = True if self.iso == 'WECC' or self.iso == 'CAISO' else False
        self.daylight_saving =  self.get_daylight_saving(period) 
        self.holidays = self.holidays(period)
        self.weekends = self.get_weekend_days()
        self.number_of_offpeak_days = self.get_number_of_offpeak_days()
        self.flat_hours = self.duration * 24 + self.daylight_saving
        self.hours = self.hours()
        
    def daylight_saving_dates(self, period):

        if calendar.Calendar(0).monthdatescalendar(int(period[:4]), 3)[0][-1] == 2:
            sunMar = calendar.Calendar(0).monthdatescalendar(int(period[:4]), 3)[2][-1]
        else:
            sunMar = calendar.Calendar(0).monthdatescalendar(int(period[:4]), 3)[1][-1]
        
        if calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[0][-1] == 10:
            sunNov = calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[1][-1]
        else:
            sunNov = calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[0][-1]
            
        return sunMar, sunNov
    
    def get_daylight_saving(self,period):
        
        Mar, Nov = self.daylight_saving_dates(period)
        
        if period[-1] == 'A' or self.iso == 'MISO' or period[-2:] == 'Q2' or period[-2:] == 'Q3':
            return 0
        elif period[-3:] == 'Mar' or  period[-2:] == 'Q1' or self.start_date == Mar:
            return -1
        elif period[-3:] == 'Nov' or  period[-2:] == 'Q4' or self.start_date == Nov:
            return 1
        else:
            return 0
        
    def hours(self):
        
        if self.peak_type == 'flat':
            return self.flat_hours
        elif self.peak_type == 'onpeak':
            return 16 * (self.duration - self.number_of_offpeak_days)
        elif self.peak_type == 'offpeak':
            return self.flat_hours - 16 * (self.duration - self.number_of_offpeak_days)
        elif self.peak_type == '7x8':
            return self.duration * 8 + self.daylight_saving
        else:
            return self.number_of_offpeak_days * 16
            
        
    def holidays(self, period):
        days = []
        #New year
        if datetime.date(int(period[:4]),1,1).weekday()== 6:
            days.append(datetime.date(int(period[:4]),1,2))
        else:
            days.append(datetime.date(int(period[:4]),1,1))
        # Memorial
        days.append(calendar.Calendar(0).monthdatescalendar(int(period[:4]), 5)[-1][0])
        # Independent
        if datetime.date(int(period[:4]),7,4).weekday() == 6:
            days.append(datetime.date(int(period[:4]),7,5))
        elif datetime.date(int(period[:4]),7,4).weekday() == 5:
            days.append(datetime.date(int(period[:4]),7,3))
        else:
            days.append(datetime.date(int(period[:4]),7,4))
        # Labor
        if calendar.Calendar(0).monthdatescalendar(int(period[:4]), 9)[0][0].month == 8:
            days.append(calendar.Calendar(0).monthdatescalendar(int(period[:4]), 9)[1][0])
        else:
            days.append(calendar.Calendar(0).monthdatescalendar(int(period[:4]), 9)[0][0])
        # Thanksgiving
        if calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[4][3].month == 12:
            days.append(calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[3][3])
        else:
            days.append(calendar.Calendar(0).monthdatescalendar(int(period[:4]), 11)[4][3])
        #Chrismas
        if datetime.date(int(period[:4]),12,25).weekday() == 5:
            days.append(datetime.date(int(period[:4]),12,24))
        elif datetime.date(int(period[:4]),12,25).weekday() == 6:
            days.append(datetime.date(int(period[:4]),12,26))
        else:
            days.append(datetime.date(int(period[:4]),12,25))
        return days
    
    def get_weekend_days(self):
        days = []
        for i in range(self.duration):
            if (self.start_date + datetime.timedelta(i)).weekday() == 6:
                days.append(self.start_date + datetime.timedelta(i))
            elif (self.start_date + datetime.timedelta(i)).weekday() == 5 and self.is_western == False:
                days.append(self.start_date + datetime.timedelta(i))
        return days
    
    def get_number_of_offpeak_days(self):
        
        number_of_days = 0
        offdays = []
        for i in self.holidays:
            offdays.append(i.strftime('%Y-%m-%d'))
        for i in self.weekends:
            offdays.append(i.strftime('%Y-%m-%d'))
        for i in range(self.duration):
            if (self.start_date + datetime.timedelta(i)).strftime('%Y-%m-%d') in offdays:
                number_of_days += 1
        return number_of_days
    
    def get_hours(self):
        print([self.iso,self.peak_type, self.start_date.strftime('%Y-%m-%d'), self.end_date.strftime('%Y-%m-%d'), self.hours])

In [410]:
cc = PowerCalendar(iso = "CAISO", peak_type = 'offpeak', period = '2021A')

In [411]:
cc.get_hours()

['CAISO', 'offpeak', '2021-01-01', '2021-12-31', 3848]


In [367]:
cc.holidays

[datetime.date(2021, 1, 1),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 7, 4),
 datetime.date(2021, 9, 6),
 datetime.date(2021, 11, 25),
 datetime.date(2021, 12, 24)]

In [368]:
cc.weekends

[datetime.datetime(2021, 1, 3, 0, 0),
 datetime.datetime(2021, 1, 10, 0, 0),
 datetime.datetime(2021, 1, 17, 0, 0),
 datetime.datetime(2021, 1, 24, 0, 0),
 datetime.datetime(2021, 1, 31, 0, 0),
 datetime.datetime(2021, 2, 7, 0, 0),
 datetime.datetime(2021, 2, 14, 0, 0),
 datetime.datetime(2021, 2, 21, 0, 0),
 datetime.datetime(2021, 2, 28, 0, 0),
 datetime.datetime(2021, 3, 7, 0, 0),
 datetime.datetime(2021, 3, 14, 0, 0),
 datetime.datetime(2021, 3, 21, 0, 0),
 datetime.datetime(2021, 3, 28, 0, 0)]

In [356]:
number_of_days = 0
for i in range(cc.duration):
    if (cc.start_date + datetime.timedelta(i) in cc.holidays)or (cc.start_date + datetime.timedelta(i) in cc.weekends):
        number_of_days +=1
number_of_days

13

In [345]:
cc.duration

90

In [359]:
cc.start_date + datetime.timedelta(0)

datetime.datetime(2021, 1, 1, 0, 0)